# Exploring and Processing the Data

In this notebook, we will begin analyzing the data. The dataset of focus here is a dataset of musical instrument reviews from Amazon. The end goal will be to categorize them as good, bad, or neutral.

In [10]:
#import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [33]:
df = pd.read_json('../data/Musical_Instruments_5.json', lines=True)

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


## Data Definition

In [37]:
df.describe()

,overall,unixReviewTime
count,10261.000000,1.026100e+04
mean,4.488744,1.360606e+09
std,0.894642,3.779735e+07
min,1.000000,1.095466e+09
25%,4.000000,1.343434e+09
50%,5.000000,1.368490e+09
75%,5.000000,1.388966e+09
max,5.000000,1.405987e+09


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      10261 non-null  object
 1   asin            10261 non-null  object
 2   reviewerName    10234 non-null  object
 3   helpful         10261 non-null  object
 4   reviewText      10261 non-null  object
 5   overall         10261 non-null  int64 
 6   summary         10261 non-null  object
 7   unixReviewTime  10261 non-null  int64 
 8   reviewTime      10261 non-null  object
dtypes: int64(2), object(7)
memory usage: 721.6+ KB


So there are no null values in all columns except the reviewerName column. As expected, the overall column is of type int. The unixReviewTime is in seconds, and of type int, but this column will likely not be of any use. From the describe method above, it looks like most rerviews are positive. This may cause issues later.

The columns with explanations are as follows:
 - reviewerID: the ID of the reviewer
 - asin: the ID of the product
 - reviewerName: the name of the reviewer
 - helpful: helpfulness rating of the review
 - reviewText: the actual review, in longform text
 - overall: the rating of the product
 - summary: a quick summary of the text review
 - unixReviewTime: the time of the review, in seconds from Jan 1 1970
 - reviewTime: the time of the review, in datetime format

Let's now clean the data and explore it.

## Data Cleaning and Exploring

In [51]:
df[df['reviewerName'].isna()].head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
41,AA5TINW2RJ195,B000068NW5,NaN,"[0, 0]",Good quality cable and sounds very good,5,Five Stars,1405382400,"07 15, 2014"
263,A14VAT5EAX3D9S,B0002CZSJO,NaN,"[0, 0]",It is exactly what you need in a capo! You can...,5,Great,1364860800,"04 2, 2013"
264,A2RVY2GDMZHH4,B0002CZSJO,NaN,"[0, 0]",Love these capos and nice that you can get the...,5,Always great Kyser,1356652800,"12 28, 2012"
432,A3BMYEA3J6RBVV,B0002CZVB4,NaN,"[0, 0]",Bought it as a gift. Friend loved it very much.,5,Friend loved it very much,1405209600,"07 13, 2014"
667,AA5TINW2RJ195,B0002D01K4,NaN,"[0, 0]",At first it look small for my guitar but it fi...,5,Love it!,1405382400,"07 15, 2014"


In [56]:
print('Number of missing reviewerNames:', df[df['reviewerName'].isna()].shape[0])

Number of missing reviewerNames: 27


It looks like there are not many reviews that are missing names, when considering that the data has 10261 reviews. This means we could comfortably remove these reviews, but because we are doing sentiment analysis and we don't care about the reviewer names at the moment, we will just leave them in.

In [63]:
print('Number of unique reviewers:', len(df['reviewerID'].unique()))
print('Number of unique instruments/products:', len(df['asin'].unique()))

Number of unique reviewers: 1429
Number of unique instruments/products: 900


So there are many repeat reviewers. This means each reviewer reviewed a product roughly 7 times. Similarly, each product was reviewed about 11 times. This is a naive way to approach the number of reviews here this way, since it was just simple division, but we can double check this by counting them now. 

We can do this by taking a using the counter function on the values of the columns. This will output a dict, with the values being the frequency of the reiviewer or instrument ID's. Taking another counter of this will let us see how many times a reviewer reviewed a product, or how many times an instrument was reviewed.

In [77]:
#seeing how many times a reviewer reviewed an instrument
Counter(Counter(df['reviewerID']).values()).most_common(5)

[(5, 558), (6, 296), (7, 188), (8, 128), (9, 71)]

In [78]:
#seeing how many times an instrument was reviewed
Counter(Counter(df['asin']).values()).most_common(5)

[(5, 197), (6, 141), (7, 107), (8, 77), (9, 61)]

So the simple division was naive indeed, as both of these cases had about 5-9 reviews only most of the time. In any case, this might be useful information later on, where we can see if a certain reviewer tends to rate more positively or not, or more importantly, if an instrument get more positive or negative reviews.